<a href="https://colab.research.google.com/github/Kwagalakwe-Grace/Text_Analysis-Project/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Comments

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for Comments: filename=comments-1.0.0-py3-none-any.whl size=2986 sha256=789042a5f6adc89616b26791aaa38addba37495769d3e3e92884c79a4929812f
  Stored in directory: /root/.cache/pip/wheels/88/f1/36/90eab26ff2097f87c6484124b498629d596b77cb7faa79e108
Successfully built Comments


In [1]:
from gc import callbacks
from google.colab import drive
import openpyxl
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataframe = openpyxl.load_workbook("/content/drive/MyDrive/course_work/Comments.xlsx")
dataframe1 = dataframe.active

In [3]:
df = pd.read_excel('/content/drive/MyDrive/course_work/Comments.xlsx')

In [4]:
df.head()

,Unnamed: 0,Comment
0,5.0,djfjkdfjkjkffdk edited
1,41.0,Faith has exhibited enthusiasm in taking on th...
2,49.0,He now has now understood the structure of gra...
3,50.0,The Intern was oriented on ICT setup and Infra...
4,52.0,The student was oriented on the organization s...


In [5]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('ADJ'):
        return wordnet.ADJ
    elif pos_tag.startswith('VERB'):
        return wordnet.VERB
    elif pos_tag.startswith('NOUN'):
        return wordnet.NOUN
    elif pos_tag.startswith('ADV'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [6]:
df.Comment=df.Comment.astype(str)

In [7]:
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_data(text):
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    text = [word for word in text if not any(c.isdigit() for c in word)]
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    text = [t for t in text if len(t) > 0]
    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    text = [t for t in text if len(t) > 1]
    text = " ".join(text)
    return(text)


In [8]:
df["cleaned_data"] = df["Comment"].apply(lambda x: clean_data(x))
df.head()

,Unnamed: 0,Comment,cleaned_data
0,5.0,djfjkdfjkjkffdk edited,djfjkdfjkjkffdk edited
1,41.0,Faith has exhibited enthusiasm in taking on th...,Faith exhibited enthusiasm taking project hand...
2,49.0,He now has now understood the structure of gra...,He understood structure grail different compon...
3,50.0,The Intern was oriented on ICT setup and Infra...,The Intern oriented ICT setup Infrastructure S...
4,52.0,The student was oriented on the organization s...,The student oriented organization structure de...


Clustering

In [9]:
dataframe = openpyxl.load_workbook("/content/drive/MyDrive/Cleaned_comments.xlsx")
dataframe2 = dataframe.active

In [10]:
corpus_1 = []
corpus_2 = []

for row in range(1, dataframe1.max_row):
    for col in dataframe1.iter_cols(2, dataframe1.max_column):
        blob = TextBlob(col[row].value)
        sentiment = blob.sentiment.polarity
        # corpus_1.append(comments(sentiment))

        score = ''
        if sentiment >= 0.80 and sentiment <= 1:
            score = "Excellent"
        if sentiment > 0 and sentiment < 0.80:
            score = "Good"
        if sentiment == 0:
            score = "Neutral"
        if sentiment < 0 and sentiment >= -0.50:
            score = "Poor"
        if sentiment < -0.50 and sentiment >= -1:
            score = "Very_Poor"

        corpus_1.append(score)
        corpus_2.append(col[row].value)

vectorizer = TfidfVectorizer(stop_words='english')
corpus_vector = vectorizer.fit_transform(corpus_1)
cluster_number = 5

vectorizer = TfidfVectorizer(analyzer='word')
X = vectorizer.fit_transform(corpus_1)

source = corpus_1
target = []
weight = []
cls = ['Excellent', 'Good', 'Neutral', 'Poor', 'Very_Poor']
df2 = pd.DataFrame(X.toarray(),columns=['excellent', 'good', 'neutral', 'poor', 'very_poor'])

class_label = {cls: i for i, cls in enumerate(cls)}
print (class_label)
print(X)

{'Excellent': 0, 'Good': 1, 'Neutral': 2, 'Poor': 3, 'Very_Poor': 4}
  (0, 2)	1.0
  (1, 1)	1.0
  (2, 2)	1.0
  (3, 3)	1.0
  (4, 1)	1.0
  (5, 2)	1.0
  (6, 2)	1.0
  (7, 1)	1.0
  (8, 2)	1.0
  (9, 1)	1.0
  (10, 1)	1.0
  (11, 3)	1.0
  (12, 1)	1.0
  (13, 1)	1.0
  (14, 2)	1.0
  (15, 2)	1.0
  (16, 1)	1.0
  (17, 1)	1.0
  (18, 2)	1.0
  (19, 2)	1.0
  (20, 1)	1.0
  (21, 1)	1.0
  (22, 1)	1.0
  (23, 2)	1.0
  (24, 2)	1.0
  :	:
  (4922, 1)	1.0
  (4923, 1)	1.0
  (4924, 1)	1.0
  (4925, 2)	1.0
  (4926, 1)	1.0
  (4927, 2)	1.0
  (4928, 1)	1.0
  (4929, 1)	1.0
  (4930, 1)	1.0
  (4931, 1)	1.0
  (4932, 2)	1.0
  (4933, 1)	1.0
  (4934, 1)	1.0
  (4935, 1)	1.0
  (4936, 3)	1.0
  (4937, 1)	1.0
  (4938, 2)	1.0
  (4939, 1)	1.0
  (4940, 1)	1.0
  (4941, 1)	1.0
  (4942, 2)	1.0
  (4943, 1)	1.0
  (4944, 2)	1.0
  (4945, 2)	1.0
  (4946, 2)	1.0


In [13]:
print(corpus_1.count("Excellent"))

245


In [14]:
print(corpus_1.count("Good"))

2816


In [15]:
print(corpus_1.count("Neutral"))

1494


In [16]:
print(corpus_1.count("Poor"))

390


In [17]:
print(corpus_1.count("Very_Poor"))

2


In [11]:
list_cols = ['Excellent','Good','Neutral','Poor','Very_Poor']
data_list = []
for i in list_cols:
  list1 = []
  list1.append(i)
  list1.append(corpus_1.count(i))
  pct = ((corpus_1.count(i))/len(corpus_1)) * 100
  list1.append(pct)
  data_list.append(list1)
data_list

to_csv_dataframe = pd.DataFrame(data_list,columns=['Cluster','Count','Percentage'])
to_csv_dataframe.to_csv('Pie_Chart.csv',index=False)